## Treinando modelos de aprendizado de máquina com a biblioteca CapyMOA

CapyMOA é uma biblioteca desenvolvida para aprendizado de máquina com stream de dados (aprendizado contínuo) que integra funcionalidades das bibliotecas MOA (stream learners), PyTorch (redes neurais) e scikit-learn (aprendizado de máquina).


#### Por que utilizar?

Tratando um dado por vez (instância), o formato de aprendizado continuo traz algumas vantagens valiosas principalmente para o contexto de redes, dentre elas: economia de memória e armazenamento, tratamento e atualização de dados em tempo real, escalabilidade, adaptação automática a mudanças (concept drift)... 

Por sua vez, o CapyMOA oferece a vantagem de ser rápido e a possibilidade de desenvolver com baixo nível de código.

#### Preparando o ambiente

O CapyMOA possui algumas dependências para ser executado:

- JAVA
- Biliotecas Python: torch & torchvision 

\# Verifica se JAVA está instalado e a versão
java -version

\# Instalação do pytorch
pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu

\# Instalação da biblioteca
pip install capymoa


In [ ]:
import capymoa

#### Preparando dados

A biblioteca possui um módulo chamado "capymoa.stream" com alguns métodos úteis para importação de dados de um dataset ou conversão de dados em uma instância.

Stream -> Retorna um stream de dados a partir de um conjunto de dados em memória, como uma lista de dicionarios ou pandas.DataFrame
Schema -> Cria uma definição de Schema à partir de propriedades
ARFFStream -> Retorna um stream de dados a partir de um arquivo .arff
CSVStream -> Retorna um stream de dados a partir de um arquivo .csv que é convertido em .arff
NumpyStream -> Retorna um stream de dados a partir de um np.array
ConcatStream -> Retorna um stream de dados concatenado a partir de dois streams menores


Como já dito, a biblioteca é desenhada para tratar uma instância por vez. Uma instância nada mais é que um novo bloco de dados, uma linha em um dataset. Por tanto, stream de dados são iteráveis e cada itter é representa uma instância.

O CApyMOA também possui ainda vários datasets de exemplo que podem ser importados a partir do módulo capymoa.datasets (Bike, CovtFD, Covtype, CovtypeNorm, CovtypeTiny, Electricity, ElectricityTiny, Sensor). Quando uma dessas classes é instânciada, um arquivo .arff será gerado dentro de uma pasta "data" no diretório atual.

Quando trabalhando com arquivos ARFF, o schema do dataset já é pré-definido. Em outros casos, é feita uma detecção automática do tipo de dado em cada coluna, porém pode ser necessária a definição manual através de uma instância de capymoa.stream.Schema utilizando o método "from_custom"

Também é possível inicializar uma instância individual, para isso utiliza-se o método "capymoa.instance.from_array" que recebe um schema e um np.array.

In [ ]:
from capymoa.stream import Schema
from capymoa.instance import Instance
import numpy as np

example_schema = Schema.from_custom(
    feature_names=["attrib_1", "attrib_2"],
    dataset_name="Hello World Schema",
    target_attribute_name="class",
    values_for_class_label=["yes", "no"]
)

example_instance_1 = Instance.from_array(
    schema=example_schema,
    instance=np.array([1.0, 2.0]),
)

example_instance_2 = Instance.from_array(
    schema=example_schema,
    instance=np.array([3.0, 4.0]),
)

print(example_schema)
print("\n", example_instance_1)
print("\n", example_instance_2)

In [ ]:
# Importa datasets
from capymoa.datasets import Sensor 
    # CovtFD, \
    # Covtype, \
    # CovtypeNorm, \
    # CovtypeTiny, \
    # Electricity, \
    # ElectricityTiny, \
    # Fried, \
    # FriedTiny, \
    # Hyper100k, \
    # RBFm_100k, \
    # RTG_2abrupt, \
    # Sensor


sensor_stream = Sensor()

sensor_stream.schema

In [ ]:
from capymoa.stream import CSVStream

csv_path = './data/cap52processed.csv'

stream_from_csv = CSVStream(csv_path,
                            
                            target_attribute_name= "label",
                            class_index= -1,
                            values_for_class_label= ['0', '1'],
                            dataset_name= "Labeled Traffic",

)

print(stream_from_csv.schema)

#### Classificadores e Avaliação

A biblioteca CapyMOA oferece suporte a diversos classificadores para modelos supervisionados, com exemplos detalhados disponíveis na documentação oficial. No entanto, para modelos semi-supervisionados, há apenas um classificador nativo implementado: o Open-Set Nearest-Neighbor (OSNN). 

Apesar dessa limitação, a biblioteca permite a criação e integração de novos classificadores personalizados. (https://capymoa.org/notebooks/05_new_learner.html)

Para avaliação de modelos supervisionados, temos o módulo "capymoa.evaluation.prequential_ssl_evaluation".

In [ ]:
from capymoa.ssl.classifier import OSNN
from capymoa.evaluation import prequential_ssl_evaluation
from capymoa.evaluation.visualization import plot_windowed_results

osnn = OSNN(
    schema=stream_from_csv.get_schema(),
)

results_osnn = prequential_ssl_evaluation(
    stream=stream_from_csv,
    learner=osnn,
    label_probability=0.05,
    window_size=100,
    max_instances=1000,
    restart_stream=False,
)

######################################

print(f"Accuracy: {results_osnn["cumulative"].accuracy()}\n")
display(results_osnn["windowed"].metrics_per_window())

results_osnn.learner = "OSNN"
plot_windowed_results(results_osnn, metric="accuracy")



In [ ]:
from capymoa.stream import CSVStream

csv_path = "./data/cap52processed.csv"

stream_from_csv = CSVStream(
    csv_path,
    target_attribute_name="label",
    class_index=-1,
    values_for_class_label=["0", "1"],
    dataset_name="NetInfect",
)

stream_from_csv.schema

In [ ]:
from capymoa.datasets import Electricity
from capymoa.evaluation import ClassificationEvaluator
from capymoa.classifier import OnlineBagging

elec_stream = Electricity()
ob_learner = OnlineBagging(
    schema=elec_stream.get_schema(),
    ensemble_size=5,
    # random_seed=1,
    # base_learner=None,
    # minibatch_size=None,
    # number_of_jobs=None,
)

ob_evaluator = ClassificationEvaluator(schema=elec_stream.get_schema())

for instance in elec_stream:
    prediction = ob_learner.predict(instance)
    ob_learner.train(instance)
    ob_evaluator.update(instance.y_index, prediction)

print("Accuracy:", ob_evaluator.accuracy())

#### Aprendizado contínuo com stream de dados